# Triangulation

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from time import time
from lib import misc, utils, app
from lib.calib import triangulate_points_fisheye

plt.style.use(os.path.join('..', 'configs', 'mplstyle.yaml'))

%load_ext autoreload
%autoreload 2

ROOT_DATA_DIR = os.path.join('..', 'data')

# Reconstruction Params
Define the params in the cell below. Thereafter, run all cells

In [ ]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, '2019_03_09', 'lily', 'run')

start_frame = 70
end_frame = 170 # use -1 to reconstruct up to the last frame possible

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in optimization
dlc_thresh = 0.5 # change this only if the optimization result is unsatisfactory

# Reconstruction

In [ ]:
assert os.path.exists(DATA_DIR), f'Data directory not found: {DATA_DIR}'
OUT_DIR = os.path.join(DATA_DIR, 'tri')
DLC_DIR = os.path.join(DATA_DIR, 'dlc')
assert os.path.exists(DLC_DIR), f'DLC directory not found: {DLC_DIR}'
os.makedirs(OUT_DIR, exist_ok=True)

# load video info
res, fps, tot_frames, _ = app.get_vid_info(DATA_DIR) # path to original videos
assert end_frame <= tot_frames, f'end_frame must be less than or equal to {tot_frames}'
assert end_frame != 0, f'end_frame cannot be 0'
if end_frame < 0:
    end_frame = end_frame % tot_frames + 1 # cyclic

assert 0 < start_frame < tot_frames, f'start_frame must be strictly between 0 and {tot_frames}'
assert 0 <= dlc_thresh <= 1, 'dlc_thresh must be from 0 to 1'

with open(os.path.join(OUT_DIR, 'reconstruction_params.json'), 'w') as f:
    json.dump(dict(start_frame=start_frame, end_frame=end_frame, dlc_thresh=dlc_thresh), f)
    
start_frame -= 1 # 0 based indexing
N = end_frame-start_frame

k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR, verbose=False)

dlc_points_fpaths = sorted(glob(os.path.join(DLC_DIR, '*.h5')))
assert n_cams == len(dlc_points_fpaths)

# Load Measurement Data (pixels, likelihood)
points_2d_df = utils.load_dlc_points_as_df(dlc_points_fpaths, verbose=False)
points_2d_df = points_2d_df[points_2d_df['frame'].between(start_frame, end_frame-1)]
points_2d_df = points_2d_df[points_2d_df['likelihood']>dlc_thresh] # ignore points with low likelihood

assert len(k_arr) == points_2d_df['camera'].nunique()

points_3d_df = utils.get_pairwise_3d_points_from_df(
    points_2d_df,
    k_arr, d_arr.reshape((-1,4)), r_arr, t_arr,
    triangulate_points_fisheye
)

points_3d_df['point_index'] = points_3d_df.index

# Save triangulation results

In [ ]:
markers = misc.get_markers()

positions = np.full((N, len(markers), 3), np.nan)
for i, marker in enumerate(markers):
    marker_pts = points_3d_df[points_3d_df['marker']==marker][['frame', 'x', 'y', 'z']].values
    for frame, *pt_3d in marker_pts:
        positions[int(frame)-start_frame, i] = pt_3d

app.save_tri(positions, OUT_DIR, scene_fpath, start_frame, dlc_thresh)

# Plot the cheetah!

In [ ]:
data_fpath = os.path.join(OUT_DIR, 'tri.pickle')
app.plot_cheetah_reconstruction(data_fpath, dark_mode=True)